In [1]:
import re


# Assignment 2: Analyzing large datasets with Spark.


For this assignment, you will need to make sure you're running from a PySpark docker environment I introduced in class. You can start the docker pySpark docker environment using the following command:

```
docker run --rm -p 4040:4040 -p 8888:8888 -v "%cd%":/home/jovyan/work jupyter/all-spark-notebook
```

Make sure you run the command from the directory containing this jupyter notebook and your data folder.


</b>
# WARNING: For some reason, ipynbb document didn't always sync properly when I was pushing to github. As such, please push often and make sure your incremental changes appear on GitHub.
</b>

### Part 1

The first part will use Spark to analyze the following books, which I have downloaded for you to use from Project Gutenberg. The files are saved to the data folder.

| File name | Book Title|
|:---------:|:----------|
|43.txt | The Strange Case of Dr. Jekyll and Mr. Hyde by Robert Louis Stevenson|
|84.txt | Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley |
|398.txt  | The First Book of Adam and Eve by Rutherford Hayes Platt|
|3296.txt | The Confessions of St. Augustine by Bishop of Hippo Saint Augustine|

The objective is to explore whether we can detect similarity between books within the same topic using word-based similarity. 

The task of identifying similar texts in Natural Language Processing is crucial. A naive method for determining whether two documents are similar is to treat them as collections of words (bag of words) and use the number of words they share as a proxy for their similarity. It makes sense that two books with religion as the topic (e.g.  `398.txt` and `3296.txt`) would have more words in common than a book that discusses religion and a book that discusses science fiction (e.g. books `84.txt` and `398.txt`). 

As mentioned above, we will be using Spark to analyze the data. Although Spark is not needed for such a small example, the platform would be ideal for analyzing very large collections of documents, like those often analyzed by large corporations

This part of the assignment will rely exclusively on RDDs.

### Q1. 
Start by importing Spark and making sure your environment is set up properly for the assignment.

Import the spark context necessary to load a document as an RDD; ignore any error messages

In [2]:
### Write your code here
from pyspark import SparkContext
sc = SparkContext()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/08 04:23:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print(f"Spark version is {sc.version}")

print(f"Phthon version is {sc.pythonVer}")

print(f"The name of the master is {sc.master}")

Spark version is 3.2.0
Phthon version is 3.9
The name of the master is local[*]


### Q2 

Read in the file `43.txt` as a spark RDD and save it to a variable called `book_43`
 * make sure `book_43` is of type MapPartitionsRDD, i.e.,
   * str(type(book_43)) == "<class 'pyspark.rdd.RDD'>" should return True 

In [4]:
### Write your code here
book_43 = sc.textFile('data/43.txt', minPartitions=4)
str(type(book_43)) == "<class 'pyspark.rdd.RDD'>"

True

### Q3

How many lines does `book_43` contain?
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [5]:
### Write your code here
book_43.count()

2935

### Q4 

Prior to analyzing the words contained in this book, we need to first remove the occurrences of non-alphabetical characters and numbers from the text. You can use the following function, which takes a line as input, removes digits and non-word characters, and splits it into a collection of words. 

```python
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()
```

Use the fucntion above on the variable (test_line) to see what it returns.
```python
test_line = "This is an example of that contains 234 and a dash-containing number"
```

In [6]:
### Write your code here
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()

test_line = "This is an example of that contains 234 and a dash-containing number"
clean_split_line(test_line)

['THIS',
 'IS',
 'AN',
 'EXAMPLE',
 'OF',
 'THAT',
 'CONTAINS',
 'AND',
 'A',
 'DASH',
 'CONTAINING',
 'NUMBER']

### Q5

How many words does `book_43` contain? To answer this question, you may find it useful to apply the function in a spark-fashion. 
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [7]:
### Write your code here
from operator import add

words = book_43.map(clean_split_line)
words = words.filter(lambda x: len(x) > 0)
word_list = sc.parallelize(words.reduce(add))
word_list.count()

29116

### Q6

How many of the words in book_43 are unique? Given that words can appear in lower, upper or mixed case (ex. The, THE, the), make sure you convert the words into lower case before counting them.


In [8]:
### Write your code here
unique = word_list.map(str.lower)
unique = unique.distinct()
unique.count()

4296

### Q7

* Generate an `RDD` that contains the frequency of each word in `book_43`. Call the variable `book_43_counts`. Each item in the `RDD` should be a tuple with the word as the first element of the tuple and the count as the second item of the tuple. The collection should look like the following:

[('project', 88), ("the", 1807), ... ]

* Such a collection may contain a large number of words and it would be imprudent to transfer all the words onto the same machine to display it. Instead, to explore the content of such a collection, display only the first element in your list. 

* Given the random nature of this operation, the first element element displayed may be different. The first entry for me was:
```
[('project', 88)]
```

* You can only use operations or actions to answer the question. 
* Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
* Code that uses functions such as `some_func(...)` is not allowed


In [9]:
### Write your code here
lower = word_list.map(str.lower)
book_43_counts = lower.map(lambda x: (x,1))
book_43_counts = book_43_counts.reduceByKey(lambda x,y: x+y)
book_43_counts.take(1)

[('where', 36)]

### Q8

Sort `book_43_counts` and print the 20 most frequent words in book_43. 
  * Hint: function `sortByKey` sorts a collection of tuples on the first element element of the list. You can easily change the order of the items in each element and use `sortByKey` to sort on the second item of each element in `book_43_counts`
  * You can only use operations or actions to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [10]:
### Write your code here
book_43_counts = book_43_counts.map(lambda x: x[::-1])
book_43_counts = book_43_counts.sortByKey(ascending=False)
book_43_counts = book_43_counts.map(lambda x: x[::-1])
book_43_counts.take(20)

[('the', 1807),
 ('of', 1068),
 ('and', 1043),
 ('to', 726),
 ('a', 686),
 ('i', 646),
 ('in', 485),
 ('was', 471),
 ('that', 392),
 ('he', 384),
 ('it', 378),
 ('you', 312),
 ('my', 308),
 ('with', 301),
 ('his', 285),
 ('had', 244),
 ('as', 203),
 ('for', 202),
 ('this', 195),
 ('but', 193)]

### Q9

You must have noted that the most frequent words in `book_43_counts` include stop words such as `of`, `the`, `and`, etc.

It would be inefficient to compare documents based on whether or not they contain stop words; those are common to all documents. As such, it's common to remove such stop words. The librarary `sklearn.feature_extraction` provides access to a collection of English stop words, which can be loaded using the following snippet:

```
from sklearn.feature_extraction import stop_words
stop_words.ENGLISH_STOP_WORDS
```

* Explore ENGLISH_STOP_WORDS (it's a frozen set data structure, i.e., a set that you cannot modify) by printing any 10 words from it. 
 * Hint: convert the frozen set to something you can subscript


In [11]:
### Write your code here
from sklearn.feature_extraction import _stop_words
eng_stop_words = sc.parallelize(_stop_words.ENGLISH_STOP_WORDS)
eng_stop_words.take(10)

['another',
 'therein',
 'show',
 'not',
 'within',
 'beside',
 'back',
 'throughout',
 'am',
 'we']

### Q10

Filter out the words in `book_43_counts` by removing those that appear in the ENGLISH_STOP_WORDS.
Save the results to a new variable called `book_43_counts_filtered`
  * You can only use operarations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses function such as `some_func(...)` is not allowed


In [12]:
### Write your code here
compare = eng_stop_words.collect()
book_43_counts_filtered = book_43_counts.filter(lambda x: x[0] not in compare)
book_43_counts_filtered.take(10)

[('s', 156),
 ('utterson', 131),
 ('said', 130),
 ('mr', 128),
 ('hyde', 105),
 ('jekyll', 105),
 ('gutenberg', 98),
 ('project', 88),
 ('man', 85),
 ('lawyer', 72)]

### Q11

* How many words are left in `book_43_counts_filtered` after removing the stop words

In [13]:
### Write your code here
book_43_counts_filtered.count()

4034

### Q12 

* Create a function called *process_RDD* that combines the relevant steps you proposed above to make it convenient to apply them to the remaining four books. Your function should accept an input text file path and:
 * Reads in the file as a textRDD
 * Cleans and splits the line using `clean_split_line`
 * Filters out the stop words
 * Returns a word count RDD where each item is a tuple of words and its count.
 


In [14]:
### Write your code here
from sklearn.feature_extraction import _stop_words

def process_RDD(text_file):
    #read in text file as rdd
    initial_rdd = sc.textFile(text_file, minPartitions=4)
    
    #clean and split lines
    init_words = initial_rdd.map(clean_split_line)
    init_words = init_words.filter(lambda x: len(x) > 0)
    init_word_list = sc.parallelize(init_words.reduce(add))
    
    #create tuples
    rdd_lower = init_word_list.map(str.lower)
    rdd_counts = rdd_lower.map(lambda x: (x,1))
    rdd_counts = rdd_counts.reduceByKey(lambda x,y: x+y)
    
    #filter our stop words
    eng_stop_words = sc.parallelize(_stop_words.ENGLISH_STOP_WORDS)
    compare = eng_stop_words.collect()
    rdd_filtered = rdd_counts.filter(lambda x: x[0] not in compare)
    
    return rdd_filtered
    

### Q13 

Apply the function `process_RDD` to `book_84`, `book_398` and `book_3296` and save the results to variables `book_84_counts_filtered`, `book_398_counts_filtered` and `book_3296_counts_filtered` respectively. How many distinct words does each book contain after filtering the stop words.


In [15]:
### Write your code here
book_84_counts_filtered = process_RDD('data/84.txt')
book_398_counts_filtered = process_RDD('data/398.txt')
book_3296_counts_filtered = process_RDD('data/3296.txt')

print('Count of book 84:')
print(book_84_counts_filtered.count())
print('Count of book 398:')
print(book_398_counts_filtered.count())
print('Count of book 3296:')
print(book_3296_counts_filtered.count())

Count of book 84:
7016
Count of book 398:
2421
Count of book 3296:
7293


### Q14 

We discussed how to evaluate similarity between two texts using the number of words they share. We hypothesized that books that are similar should have more words in common than books that are dissimilar. If that holds, `book_398` and `book_3296`, which both pertain to religion, will have more words in common than, say, `book_84` and `book_398`. Test this hypothesis by writing code that compares and prints the number of words shared between `book_398` and `book_3296` and then between `book_84` and `book_398`.


In [16]:
### Write your code here
book_84_words = book_84_counts_filtered.map(lambda x: x[0])
book_398_words = book_398_counts_filtered.map(lambda x: x[0])
book_3296_words = book_3296_counts_filtered.map(lambda x: x[0])

print('Words shared between book 84 and book 3296:')
print(book_84_words.intersection(book_3296_words).count())
print('Words shared between book 84 and book 398:')
print(book_84_words.intersection(book_398_words).count())

Words shared between book 84 and book 3296:
3608
Words shared between book 84 and book 398:
1691


### Q15

* Based on the above, do you think counting the number of shared words is a good idea as a distance metric for evaluating topic similarity? Justify your answer?
* Hint: What do *book_84* and *book_3296* have in common? 

No. Computing the number of shared words could be decent distance metric for topic similarity if it was modified slightly. Our hypothesis that similar books have similar words appears to hold up in this case. From here, we could compute the shared words over the total words in both books to get a normalized similarity. In this case, more similar books would have a value closer to 1, and further books would have a value closer to 0. 
Without additional modification, however, using the number of shared words may fail as a distance metric. For example, a large book covering multiple topics may have a large portion of shared words, but may not be as similar as a shorter book in the field focusing on a specific area. 

## Part II 

Another approach to estimating similarity consists of computing the Euclidean distance across a set of words. For example, suppose we have 3 documents A, B and C with the following counts for the words `evolution`, `DNA`, `biology` and `finance`. 

```python 
A = [4, 9, 6, 8]
B = [3, 7, 7, 10]
C = [15, 10, 1, 1]
```
Although all documents contain exactly the four words, the number of times these words appear in each book may be indicative of thier topic. For example, documents `A` and `B` are more likely to be business related since they contain the word `finance` more frequently (8 and 10 times respectively). Document `C` may be a technical document since it focuses on more technical words (`evolution` and `DNA`) and less on the words `finance`.

The Euclidean distance, which can be computed using the `scikit` snippet below, is more indicative of topic-relatedness between the two documents.

```python
from scipy.spatial.distance import euclidean 
print(f"The Euclidean distance between A and B is: {euclidean(A, B)}")

print(f"The Euclidean distance between A and C is: {euclidean(A, C)}")

print(f"The Euclidean distance between B and C is: {euclidean(B, C)}")
```


### Q16

To calculate the Euclidean distance, we must first identify the set of words by which we will compare the documents. Here, we will explore the words that are common to all 4 documents. We will store the data in a matrix called `counts_matrix`.

Start by finding the words that are common to all four documents after stop-word filtering and store the counts for each word in a column of `counts_matrix`. 

To take the previous example, you can generate an emtpy matrix with 3 lines (documents `A`, `B` and `C`) and 4 columns (words `evolution`, `DNA`, `biology` and `finance`) using the following code.

```python
import numpy as np
counts_matrix = np.zeros([3,4])
```

After generting the counts, you can fill the counts for a document, say `A`, using the following code:

```python
counts_matrix[0, :] = [4, 9, 6, 8] 
```
* Other than for building `counts_matrix` you should exclusively use operations or actions on the `RDD` to answer this question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [17]:
### Write your code here
import numpy as np
#find total intersection
book_43_words = book_43_counts_filtered.map(lambda x: x[0])
total_intersection = book_43_words.intersection(book_84_words)
total_intersection = total_intersection.intersection(book_3296_words)
total_intersection = total_intersection.intersection(book_398_words)

#create subset of rdds with intersection
intersection = total_intersection.collect()
book_43_counts_intersection = book_43_counts_filtered.filter(lambda x: x[0] in intersection)
book_84_counts_intersection = book_84_counts_filtered.filter(lambda x: x[0] in intersection)
book_3296_counts_intersection = book_3296_counts_filtered.filter(lambda x: x[0] in intersection)
book_398_counts_intersection = book_398_counts_filtered.filter(lambda x: x[0] in intersection)

#order sorted rdds so that they are the same
book_43_counts_intersection = book_43_counts_intersection.sortByKey()
book_84_counts_intersection = book_84_counts_intersection.sortByKey()
book_3296_counts_intersection = book_3296_counts_intersection.sortByKey()
book_398_counts_intersection = book_398_counts_intersection.sortByKey()

#create the array
counts_matrix = np.zeros([4, total_intersection.count()])
counts_matrix[0, :] = book_43_counts_intersection.map(lambda x: x[1]).collect()
counts_matrix[1, :] = book_84_counts_intersection.map(lambda x: x[1]).collect()
counts_matrix[2, :] = book_3296_counts_intersection.map(lambda x: x[1]).collect()
counts_matrix[3, :] = book_398_counts_intersection.map(lambda x: x[1]).collect()

#display, to check
counts_matrix[0:4, 0:12]

array([[ 1.,  3.,  2.,  2.,  1., 10.,  1.,  1.,  2.,  1.,  2.,  2.],
       [ 1.,  9.,  4.,  2.,  1., 12.,  1.,  1.,  2.,  3., 21.,  4.],
       [16., 36.,  3.,  3.,  1., 12.,  1.,  1.,  2., 39., 15.,  1.],
       [ 1.,  5., 16., 14.,  1., 10.,  1.,  1.,  2.,  9., 11.,  2.]])

### Q17

Compute the Euclidean distance between `book_398` and `book_3296`, which both talk about religion and `book_84` and `book_398`. What do you conclude about using the Euclidean distance for evaluating topic relatedness across documents?


In [18]:
### Write your code here
from scipy.spatial.distance import euclidean 

print(f"The Euclidean distance between book 398 and book 3296 is: {euclidean(counts_matrix[3, :], counts_matrix[2, :])}")

print(f"The Euclidean distance between book 84 and book 398 is: {euclidean(counts_matrix[3, :], counts_matrix[1, :])}")

The Euclidean distance between book 398 and book 3296 is: 1156.6628722320086
The Euclidean distance between book 84 and book 398 is: 751.6688100486809


Euclidian distance also appears to be a poor metric for evaluating topic relatedness. Our two books about religon have a greater distance between them than one book about religon to an unrelated book. This could be due to a book being longer than the other, hence containing significantly more words, and thereby more occurances of shared words.

### Q18

Bonus question (5 points): Can you think of a few things we could do to improve similarity between documents that pertain to the same topic. Justify your answer without giving code

We could normalize the word counts. It may be that similar documents have a similar proportion of words pertaining to the subject matter, and normalizing the word counts would prevent large books from being too different from smaller books. 
Alternatively, we could use the total word space of all documents we are comparing with 0s representing words not present in the book. That may increase distance between dissimilar books.

## Part III

In this part we will build some basic analytics for a dataset consisting of flight arrival and departure details for all commercial flights within the USA in one month. While this dataset can be managed using Pandas (<1M records), scaling to a yearly or longer timeframe will greatly benefit from using a distributed computing framework such as `Spark`.

Here, you should use exclusively `SparkDatFrames. 

We want to analyze this dataset to better schedule trips.  For example:
 * Avoid airlines carriers that are most often associated with delays.
 * Avoid departure days where delays are most frequent.
 * Avoid airports which are associated with delays or long taxxying time.
* etc.
 

The information about the fields contained in the data file can be found [here](https://dataverse.harvard.edu/dataset.xhtml;jsessionid=0414e25969eccd0e88ae4d64fa0b?persistentId=doi%3A10.7910%2FDVN%2FHG7NV7&version=&q=&fileTypeGroupFacet=&fileTag=%221.+Documentation%22&fileSortField=date&fileSortOrder=desc)


### Q19

Load the file `flight_info.csv` into a spark `DataFrame` called `fight_info`.

  * Note that you will need to create a sparkSession prior to loading the data
  
* How many entries does the file contain?



In [19]:
#spark session creation 
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL flight info") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [20]:
### Write your code here
fight_info = spark.read.option("header", True).csv('data/flight_info.csv')
fight_info.count()

450017

### Q20

Use `pySpark-SQL` or `pandas-like syntax to compute the airlines represented in this dataset
The airline information is stored in a field called UniqueCarrier
* UniqueCarrier: Represents the unique carrier code (ex.AA = American Airlines) 


In [21]:
### Write your code here
fight_info.createOrReplaceTempView("flights")

unique_carriers = spark.sql("SELECT DISTINCT UniqueCarrier FROM flights")
unique_carriers.show()

+-------------+
|UniqueCarrier|
+-------------+
|           AA|
|           EV|
|           B6|
|           DL|
|           UA|
|           NK|
|           OO|
|           F9|
|           HA|
|           WN|
|           AS|
|           VX|
+-------------+



### Q21

The data file contains various other fields, two of which are useful for answering the next question.

* CRSDepTime: Represents the scheduled departure time
* DepTime: Represents the actual departure time

Compute the number of flights delayed per each carried code represented in this dataset. Sort the data by decreasing order of delays.
  * A delay is observed when `DepTime` > `CRSDepTime`


In [22]:
### Write your code here
delays = spark.sql("SELECT UniqueCarrier, (DepTime - CRSDepTime) AS Delay FROM flights ORDER BY Delay DESC")

delays = delays.na.drop()
delays = delays.where(delays['Delay'] > 0)
delays.groupby('UniqueCarrier').count().orderBy('count', ascending=False).show()

+-------------+-----+
|UniqueCarrier|count|
+-------------+-----+
|           WN|47472|
|           DL|24334|
|           AA|23461|
|           UA|17701|
|           OO|16751|
|           EV|11596|
|           B6| 9396|
|           AS| 4488|
|           NK| 4151|
|           F9| 2988|
|           VX| 2648|
|           HA| 1939|
+-------------+-----+



### Q22

 Use the file `airlines.csv` to find the the complete name of the airline. Here, you are required to load the file as a pyspark DataFrame; call it `airlines_info`, and repeat the query above while including the `flights.csv `file in your query ( requires doing a `join`) so that you can also display the full name of the carrier (second column). 

The result will look (approximately) like:

```
[Row(UniqueCarrier='WN', first(_c1)='Southwest Airlines', count=SOME_count),
 Row(UniqueCarrier='DL', first(_c1)='Delta Air Lines', count=SOME_count),
 Row(UniqueCarrier='AA', first(_c1)='American Airlines', count=SOME_count),
 ...
 ]
```

The carrier code in the `airlines.csv` file is provided in the 4th (1-based) column

Note that the file `airlines.csv` does not have column header. Hence, you need to print one line of your dataset to see what names Spark gave to the columns. Use the name provided by Spark in your query.

In [23]:
### Write your code here
airlines_info = spark.read.csv('data/airlines.csv')
airlines_info.show(3)

airlines_info.createOrReplaceTempView("airlines")
delays.createOrReplaceTempView("delays")

+---+--------------+---+----+---+-------+-------------+---+
|_c0|           _c1|_c2| _c3|_c4|    _c5|          _c6|_c7|
+---+--------------+---+----+---+-------+-------------+---+
|  1|Private flight| \N|   -|N/A|   null|         null|  Y|
|  2|   135 Airways| \N|null|GNL|GENERAL|United States|  N|
|  3| 1Time Airline| \N|  1T|RNX|NEXTIME| South Africa|  Y|
+---+--------------+---+----+---+-------+-------------+---+
only showing top 3 rows



In [24]:
delays_name = spark.sql("SELECT UniqueCarrier, _c1 from delays JOIN airlines ON delays.UniqueCarrier = airlines._c3")
delays_name.groupby(['UniqueCarrier', '_c1']).count().orderBy('count', ascending=False).show()

+-------------+--------------------+-----+
|UniqueCarrier|                 _c1|count|
+-------------+--------------------+-----+
|           WN|  Southwest Airlines|47472|
|           DL|     Delta Air Lines|24334|
|           AA|   American Airlines|23461|
|           UA|     United Airlines|17701|
|           OO|             SkyWest|16751|
|           EV|Atlantic Southeas...|11596|
|           B6|     JetBlue Airways| 9396|
|           AS|     Alaska Airlines| 4488|
|           NK|     Spirit Airlines| 4151|
|           F9|   Frontier Airlines| 2988|
|           VX|      Virgin America| 2648|
|           HA|   Hawaiian Airlines| 1939|
+-------------+--------------------+-----+



### Q23

Compute the number of delays per company per day. The day is encoded as an integer in the column `DayOfWeek` in `fight_info`. You can display the day as an integer or map it into a string name of the week.
Sort the data by airline code (UniqueCarrier) and by increasing values of DayOfWeek


You results should look like the following



In [35]:
### Write your code here
delays_day = spark.sql("SELECT UniqueCarrier, _c1, DayOfWeek from flights JOIN airlines ON flights.UniqueCarrier = airlines._c3")
delays_day.groupby(['UniqueCarrier', '_c1', 'DayOfWeek']).count().orderBy('count', ascending=False).show()

+-------------+------------------+---------+-----+
|UniqueCarrier|               _c1|DayOfWeek|count|
+-------------+------------------+---------+-----+
|           WN|Southwest Airlines|        1|18303|
|           WN|Southwest Airlines|        2|18123|
|           WN|Southwest Airlines|        7|16463|
|           WN|Southwest Airlines|        3|14409|
|           WN|Southwest Airlines|        5|14400|
|           WN|Southwest Airlines|        4|14363|
|           DL|   Delta Air Lines|        1|12290|
|           AA| American Airlines|        1|12165|
|           WN|Southwest Airlines|        6|11724|
|           AA| American Airlines|        7|11678|
|           AA| American Airlines|        2|11543|
|           DL|   Delta Air Lines|        2|11515|
|           DL|   Delta Air Lines|        7|10615|
|           AA| American Airlines|        5| 9944|
|           AA| American Airlines|        4| 9894|
|           DL|   Delta Air Lines|        5| 9724|
|           AA| American Airlin

### Q24  

Counting the number of delayed flights per airline is misleading, as airlines with more flights are more likley to have delays than companies with substantially fiewer flights. 

Repeat the same query above but, for each carrier, normalize the counts of delays by the total number of flights for that carrier. 


In [66]:
### Write your code here
all_flights = spark.sql("SELECT UniqueCarrier FROM flights")
flight_totals = all_flights.groupby('UniqueCarrier').count().orderBy('count', ascending=False)
flight_delays = delays_name.groupby(['UniqueCarrier', '_c1']).count().orderBy('count', ascending=False)

flight_totals.createOrReplaceTempView("ftots")
flight_delays.createOrReplaceTempView("fdels")

normal_delays = spark.sql("SELECT fdels.UniqueCarrier, _c1, (fdels.count/ftots.count) AS NormalDelays FROM ftots JOIN fdels on ftots.UniqueCarrier == fdels.UniqueCarrier")
normal_delays.orderBy('NormalDelays', ascending=False).show()

+-------------+--------------------+-------------------+
|UniqueCarrier|                 _c1|       NormalDelays|
+-------------+--------------------+-------------------+
|           VX|      Virgin America| 0.4579730197163611|
|           WN|  Southwest Airlines| 0.4404323421626386|
|           UA|     United Airlines| 0.4174468787585784|
|           F9|   Frontier Airlines| 0.3850515463917526|
|           B6|     JetBlue Airways|0.38192016909194376|
|           DL|     Delta Air Lines|0.34855972383366995|
|           OO|             SkyWest|0.33404458979779045|
|           EV|Atlantic Southeas...| 0.3309644090532865|
|           NK|     Spirit Airlines|  0.330230708035004|
|           AA|   American Airlines|0.32080347864136083|
|           HA|   Hawaiian Airlines|0.30895474824729124|
|           AS|     Alaska Airlines|0.30507783291414586|
+-------------+--------------------+-------------------+



### Q25 

Time the query above. How long did it take to run. 
  * Make sure you run the code a few times and compute the average run time.
  * The above should be easy to implement if you use the correct Jupyter Notebook `magic` function
  

In [68]:
%%timeit
all_flights = spark.sql("SELECT UniqueCarrier FROM flights")
flight_totals = all_flights.groupby('UniqueCarrier').count().orderBy('count', ascending=False)
flight_delays = delays_name.groupby(['UniqueCarrier', '_c1']).count().orderBy('count', ascending=False)

flight_totals.createOrReplaceTempView("ftots")
flight_delays.createOrReplaceTempView("fdels")

normal_delays = spark.sql("SELECT fdels.UniqueCarrier, _c1, (fdels.count/ftots.count) AS NormalDelays FROM ftots JOIN fdels on ftots.UniqueCarrier == fdels.UniqueCarrier")

31.2 ms ± 2.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Q26 

Use one of the techniques covered in class to accelerate this query. Time your query to see by how much the run time was improved

In [28]:
### Write your code here

### Q27 

Is the departure delay (i.e., DepTime - CRSDepTime) predictive of the arrival delay (ArrTime > CRSArrTime)?
Use an approach of your choice (e.g. `skelearn` which we covered in class or `Spark`) to model as a linear regression the arrival delay as a function of the departure delay. 



In [29]:
### Write your code here